In [42]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures


In [30]:
df=pd.read_csv('Fraud.csv')


In [31]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [32]:
df.drop(['nameOrig','nameDest'],axis=1,inplace=True)
df.columns

Index(['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud'],
      dtype='object')

In [33]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   oldbalanceOrg   float64
 4   newbalanceOrig  float64
 5   oldbalanceDest  float64
 6   newbalanceDest  float64
 7   isFraud         int64  
 8   isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 436.9+ MB


In [34]:
from sklearn.preprocessing import LabelEncoder

la=LabelEncoder()
df['type']=la.fit_transform(df['type'])

In [35]:
df['type'].value_counts()


1    2237500
3    2151495
0    1399284
4     532909
2      41432
Name: type, dtype: int64

In [36]:
Train=df.drop('isFraud',axis=1)
Test=df['isFraud']

In [37]:
#TRAIN/TEST/CV SETS
X_train, X_test, Y_train, Y_test = train_test_split(Train, Test,
    test_size=0.2,random_state = 0)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, 
    test_size=0.25, random_state= 0) # 0.25 x 0.8 = 0.2
print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(Y_train.shape))
print("y_test shape: {}".format(Y_test.shape))
print("X_val shape: {}".format(X_val.shape))
print("y val shape: {}".format(Y_val.shape))

X_train shape: (3817572, 8)
X_test shape: (1272524, 8)
y_train shape: (3817572,)
y_test shape: (1272524,)
X_val shape: (1272524, 8)
y val shape: (1272524,)


In [40]:
################### Degree 1 ################
lr_clf=LogisticRegression()
lr_clf.fit(X_train,Y_train)
lr_pred=lr_clf.predict(X_test)
print("Accuracy : {:0.4f}".format(accuracy_score(Y_test,lr_pred)))
print("Coffiecients: {}".format(lr_clf.coef_))
print("Intercept: {}".format(lr_clf.intercept_))

Accuracy : 0.9982
Coffiecients: [[-2.72491305e-02 -3.18527044e-04 -2.03055078e-04  2.22256435e-04
  -2.42207643e-04  7.16247541e-06 -8.47105281e-06  8.36010513e-09]]
Intercept: [-0.00011281]


In [44]:
################### Degree 2 ################
poly_features = PolynomialFeatures(degree=2)
# transform the features to higher degree features.
X_train_poly = poly_features.fit_transform(X_train)
# fit the transformed features to Linear Regression
poly_model = LogisticRegression()
poly_model.fit(X_train_poly, Y_train)   
# predicting on test data-set
y_test_predicted = poly_model.predict(poly_features.fit_transform(X_test))
print("Accuracy : {:0.4f}".format(accuracy_score(Y_test,y_test_predicted)))
print('Coefficients: {}'.format(lr_clf.coef_))
print('Intercept: {}'.format(lr_clf.intercept_))

C:\Users\karim tamer\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy : 0.9987
Coefficients: [[-2.72491305e-02 -3.18527044e-04 -2.03055078e-04  2.22256435e-04
  -2.42207643e-04  7.16247541e-06 -8.47105281e-06  8.36010513e-09]]
Intercept: [-0.00011281]


In [ ]:
################### Degree 3 ################
poly_features = PolynomialFeatures(degree=3)
# transform the features to higher degree features.
X_train_poly = poly_features.fit_transform(X_train)
# fit the transformed features to Linear Regression
poly_model = LogisticRegression()
poly_model.fit(X_train_poly, Y_train)   
# predicting on test data-set
y_test_predicted = poly_model.predict(poly_features.fit_transform(X_test))
print("Accuracy : {:0.4f}".format(accuracy_score(Y_test,y_test_predicted)))
print('Coefficients: {}'.format(lr_clf.coef_))
print('Intercept: {}'.format(lr_clf.intercept_))